# Hidden Markov Model

## 1. Import Libraries

In [1]:
import numpy as np

from hmmlearn import hmm # https://hmmlearn.readthedocs.io/en/latest/
from pylearn_ml191.hmm import HMM

# Fix random seed
np.random.seed(21)

In [2]:
# Helper functions
#-------------------
def keys_to_values(d, keys):
    return [d[k] for k in keys]

def values_to_keys(d, values):
    def get_index(d, val):
        for k, v in d.items():
            if v == val:
                return k
            
    return [get_index(d, v) for v in values]

## 2. Solutions for three problems
Ref: https://www.ece.ucsb.edu/Faculty/Rabiner/ece259/Reprints/tutorial%20on%20hmm%20and%20applications.pdf

In [3]:
obs_space_dict = { "Đỏ": 0, "Xanh": 1, "Vàng": 2}
state_dict =  {"Can 1": 0, "Can 2": 1, "Can 3": 2}

A = np.array([
    [0.1, 0.5, 0.4],
    [0.3, 0.3, 0.4],
    [0.4, 0.4, 0.2]
])

B = np.array([
    [0.2, 0.25, 0.55],
    [0.45, 0.2, 0.35],
    [0.2, 0.1, 0.7],
])

pi = np.array([0.3, 0.3, 0.4])

In [4]:
model = HMM(A, B, pi)
print("Model from pylearn_ml191: ")
print("=========================")
model.show_model()

Model from pylearn_ml191: 
A: Transition probability matrix
[[0.1 0.5 0.4]
 [0.3 0.3 0.4]
 [0.4 0.4 0.2]]
------------------------------
B: Emission probability matrix
[[0.2  0.25 0.55]
 [0.45 0.2  0.35]
 [0.2  0.1  0.7 ]]
-------------------------------
pi: Initital state distribution
[0.3 0.3 0.4]


In [5]:
model_hmmlearn = hmm.MultinomialHMM(n_components=3, 
                           algorithm="viterbi",
                           random_state=21,
                           n_iter=10000,
                           tol=0.05)

# Specify pi, A, B of model 
model_hmmlearn.transmat_ = A
model_hmmlearn.emissionprob_ = B
model_hmmlearn.startprob_ = pi

print("Model from hmmlearn: ")
print("====================")
print("A: Transition probability matrix")
print(model_hmmlearn.transmat_)
print("B: Emission probability matrix")
print(model_hmmlearn.emissionprob_)
print("pi: Initital state distribution")
print(model_hmmlearn.startprob_)

Model from hmmlearn: 
A: Transition probability matrix
[[0.1 0.5 0.4]
 [0.3 0.3 0.4]
 [0.4 0.4 0.2]]
B: Emission probability matrix
[[0.2  0.25 0.55]
 [0.45 0.2  0.35]
 [0.2  0.1  0.7 ]]
pi: Initital state distribution
[0.3 0.3 0.4]


### 2.1 Problem 1

In [7]:
obs_seq = ["Vàng", "Xanh"]
obs_idxs = np.array(keys_to_values(obs_space_dict, obs_seq))

print("=======================================================")
print("** PROBLEM 1: Calc P(O1, O2, O3, ... , OT | pi, A, B) ")
print("=======================================================")
print("Observation Sequences: ", obs_seq)
print("=======================================================")
print("Result from pylearn_ml191: ")
print("Prob(O | pi, A, B) = {:0.4f}".format(model.score(obs_idxs)))
print("-------------------------------------------------------")
print("Result from hmmlearn: ")
print("Prob(O | pi, A, B) = {:0.4f}".format(np.exp(model_hmmlearn.score(obs_idxs.reshape(-1, 1)))))
print("=======================================================")

** PROBLEM 1: Calc P(O1, O2, O3, ... , OT | pi, A, B) 
Observation Sequences:  ['Vàng', 'Xanh']
Result from pylearn_ml191: 
Prob(O | pi, A, B) = 0.1016
-------------------------------------------------------
Result from hmmlearn: 
Prob(O | pi, A, B) = 0.1016


### 2.2 Problem 2

In [8]:
print("========================================================================")
print("** PROBLEM 2: Find state seq is best 'explained' the from observations" )
print("========================================================================")
states_idx, prob = model.decode(obs_idxs)
print("Result from pylearn_ml191: ")
print ("Most likely state sequence: ", values_to_keys(state_dict, states_idx))
print("Probability: {:0.6f}".format(prob))
print("------------------------------------------------------------------------")
hmmlearn_log_prob, hmmlearn_state_idx = model_hmmlearn.decode(obs_idxs.reshape(-1, 1))
hmmlearn_prob = np.exp(hmmlearn_log_prob)
print("Result from hmmlearn: ")
print ("Most likely state sequence: ", values_to_keys(state_dict, hmmlearn_state_idx))
print("Probability: {:0.6f}".format(hmmlearn_prob))
print("========================================================================")

** PROBLEM 2: Find state seq is best 'explained' the from observations
Result from pylearn_ml191: 
Most likely state sequence:  ['Can 3', 'Can 1']
Probability: 0.028000
------------------------------------------------------------------------
Result from hmmlearn: 
Most likely state sequence:  ['Can 3', 'Can 1']
Probability: 0.028000


### 2.3 Problem 3

In [8]:
print("=============================================================")
print("** PROBLEM 3: Learn HMM - train the model to a local maximal ")
print("-------------------------------------------------------------")

print("---> Generate data for training hidden markov model ... ")
print("---> Done!")
print("Data: ")
print("-----")
observations_data, states_data = model.sample(500)
print(observations_data)
print("---> Starting learning")
print("----------------------")
# Init A, B, pi
A_init = np.array([
    [0.3, 0.3, 0.4],
    [0.1, 0.6, 0.3],
    [0.2, 0.2, 0.6]
])

B_init = np.array([
    [0.3, 0.4, 0.3],
    [0.2, 0.2, 0.6],
    [0.25, 0.25, 0.5],
])

pi_init = np.array([0.3, 0.5, 0.2])

model2 = HMM(A_init, B_init, pi_init)

model2.fit(observations_data, 
           max_step=10000, 
           stop_criterion=0.01, 
           verbose=10)

print("---> Done!")

** PROBLEM 3: Learn HMM - train the model to a local maximal 
-------------------------------------------------------------
---> Generate data for training hidden markov model ... 
---> Done!
Data: 
-----
[2 1 1 2 0 1 0 2 2 2 2 1 1 1 1 2 1 2 2 2 2 0 2 2 2 2 1 1 2 2 0 2 2 2 0 1 2
 0 2 2 2 2 2 2 2 1 0 1 2 2 1 2 1 2 2 0 2 0 0 2 2 0 0 2 2 2 2 0 2 2 0 0 1 1
 0 2 0 0 0 1 2 1 2 0 2 1 0 2 2 1 2 2 2 0 2 2 2 0 0 2 2 2 1 2 0 0 2 2 1 0 2
 1 0 1 2 0 2 0 2 1 0 0 2 0 2 2 2 2 2 0 0 2 2 2 0 2 2 2 2 2 0 2 1 1 0 0 1 1
 0 0 1 0 2 0 1 0 2 2 2 0 1 2 1 2 2 0 0 0 0 2 2 2 0 2 0 0 2 0 1 0 2 2 2 1 1
 2 2 0 1 0 2 2 2 0 2 2 1 0 2 2 2 2 2 0 0 2 1 1 0 2 0 2 1 2 1 2 2 0 2 2 2 2
 2 0 2 0 0 2 0 0 2 2 2 2 2 2 0 0 2 2 2 2 0 0 0 2 0 2 2 2 1 2 2 0 2 2 1 2 0
 0 2 2 1 0 2 2 2 0 2 0 1 2 2 2 0 2 0 2 2 2 1 2 1 0 0 2 0 0 1 0 2 1 0 0 2 2
 2 2 0 2 2 2 1 1 1 2 0 0 2 2 2 2 2 2 1 0 0 0 2 2 2 1 1 2 0 1 2 2 0 1 2 2 2
 2 2 1 1 2 2 2 0 2 0 0 2 2 1 0 2 2 2 0 0 1 1 1 0 0 1 2 2 2 2 0 2 0 2 2 2 1
 0 2 2 2 0 2 2 2 1 2 1 0 0 2 2 1 2 1 2 2 2 2 

In [9]:
# Specify pi, A, B of model 
model_hmmlearn.transmat_ = A_init
model_hmmlearn.emissionprob_ = B_init
model_hmmlearn.startprob_ = pi_init

model_hmmlearn.fit(observations_data.reshape(-1, 1))
print("Done!")

Done!


In [10]:
print("Score with hmmlearn: ")
print("--------------------")
print("P(O|model) = {:0.6f}".format(np.exp(model_hmmlearn.score(obs_idxs.reshape(-1, 1)))))

Score with hmmlearn: 
--------------------
P(O|model) = 0.001379


In [11]:
print("Score with pylearn_ml191: ")
print("--------------------")
print("P(O|model) = {:0.6f}".format(model.score(obs_idxs)))

Score with pylearn_ml191: 
--------------------
P(O|model) = 0.004657
